In [5]:
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinshi.csv')
data_rfe=np.array(data_rfe)
fea=data_rfe[:,:49]
target=data_rfe[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

# SVM-RFE_RELIEF

In [6]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,alpha,folds):
        #kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            if len(candi_fea)==optim_featur_num:
                return candi_fea
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(i)
            i-=1

In [7]:
rfe_relief_model=rfe_relief_SVM(x_train,y_train,1.5,0.1)

In [8]:
aver_mean_lis1,F_measure_lis1=rfe_relief_model.featu_score(0.15,5)
print(aver_mean_lis1,F_measure_lis1)

[0.7922542930215564, 0.7789524505454356, 0.798873897385041, 0.8049976512344068, 0.8014236129234303, 0.8024414113471476, 0.8009068844929276, 0.7947661673364997, 0.7968135080119005, 0.7963046088000418, 0.7952881152460984, 0.7932420794404719, 0.7860835638603267, 0.7763870765697585, 0.7641317396523828, 0.7702541886319745, 0.7620909233258522, 0.7626050420168068, 0.7600540216086434, 0.7523957409050577, 0.7488177879847591, 0.7513727230022444, 0.7493266871966178, 0.7508625189206117, 0.7472871757398611, 0.7472871757398611, 0.7462654627068218, 0.7483062790333526, 0.7437026984706927, 0.7452359204551385, 0.7431964089983819, 0.7391121666057727, 0.7380930633122814, 0.7370674356699201, 0.7334933973589435, 0.7345138055222089, 0.7334881778798475, 0.7273670337700299, 0.7248094890129965, 0.7253223028341772, 0.7243018946709118, 0.7268503053395271, 0.7273644240304817, 0.7268555248186231, 0.726856829688397, 0.7227686726864659, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545, 

In [9]:
aver_mean_lis2,F_measure_lis2=rfe_relief_model.featu_score(0.25,5)
print(aver_mean_lis2,F_measure_lis2)

[0.7922542930215564, 0.7789524505454356, 0.798873897385041, 0.8049976512344068, 0.8014236129234303, 0.8024414113471476, 0.8009068844929276, 0.7947661673364997, 0.7968135080119005, 0.7963046088000418, 0.7952881152460984, 0.7932420794404719, 0.7860835638603267, 0.7763870765697585, 0.7641317396523828, 0.7702541886319745, 0.7620909233258522, 0.7626050420168068, 0.7600540216086434, 0.7523957409050577, 0.7488177879847591, 0.7513727230022444, 0.7493266871966178, 0.7508625189206117, 0.7472871757398611, 0.7472871757398611, 0.7462654627068218, 0.7483062790333526, 0.7437026984706927, 0.7452359204551385, 0.7431964089983819, 0.7391121666057727, 0.7380930633122814, 0.7370674356699201, 0.7334933973589435, 0.7345138055222089, 0.7334881778798475, 0.7273670337700299, 0.7248094890129965, 0.7253223028341772, 0.7243018946709118, 0.7268503053395271, 0.7273644240304817, 0.7268555248186231, 0.726856829688397, 0.7227686726864659, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545, 

In [10]:
aver_mean_lis3,F_measure_lis3=rfe_relief_model.featu_score(0.35,5)
print(aver_mean_lis3,F_measure_lis3)

[0.7922542930215564, 0.7789524505454356, 0.798873897385041, 0.8049976512344068, 0.8014236129234303, 0.8024414113471476, 0.8009068844929276, 0.7947661673364997, 0.7968135080119005, 0.7963046088000418, 0.7952881152460984, 0.7932420794404719, 0.7860835638603267, 0.7763870765697585, 0.7641317396523828, 0.7702541886319745, 0.7620909233258522, 0.7626050420168068, 0.7600540216086434, 0.7523957409050577, 0.7488177879847591, 0.7513727230022444, 0.7493266871966178, 0.7508625189206117, 0.7472871757398611, 0.7472871757398611, 0.7462654627068218, 0.7483062790333526, 0.7437026984706927, 0.7452359204551385, 0.7431964089983819, 0.7391121666057727, 0.7380930633122814, 0.7370674356699201, 0.7334933973589435, 0.7345138055222089, 0.7334881778798475, 0.7273670337700299, 0.7248094890129965, 0.7253223028341772, 0.7243018946709118, 0.7268503053395271, 0.7273644240304817, 0.7268555248186231, 0.726856829688397, 0.7227686726864659, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545, 

In [11]:
aver_mean_lis4,F_measure_lis4=rfe_relief_model.featu_score(0.45,5)
print(aver_mean_lis4,F_measure_lis4)

[0.7922542930215564, 0.7789524505454356, 0.798873897385041, 0.8049976512344068, 0.8014236129234303, 0.8024414113471476, 0.8009068844929276, 0.7947661673364997, 0.7968135080119005, 0.7963046088000418, 0.7952881152460984, 0.7932420794404719, 0.7860835638603267, 0.7763870765697585, 0.7641317396523828, 0.7702541886319745, 0.7620909233258522, 0.7626050420168068, 0.7600540216086434, 0.7523957409050577, 0.7488177879847591, 0.7513727230022444, 0.7493266871966178, 0.7508625189206117, 0.7472871757398611, 0.7472871757398611, 0.7462654627068218, 0.7483062790333526, 0.7437026984706927, 0.7452359204551385, 0.7431964089983819, 0.7391121666057727, 0.7380930633122814, 0.7370674356699201, 0.7334933973589435, 0.7345138055222089, 0.7334881778798475, 0.7273670337700299, 0.7248094890129965, 0.7253223028341772, 0.7243018946709118, 0.7268503053395271, 0.7273644240304817, 0.7268555248186231, 0.726856829688397, 0.7227686726864659, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545, 

In [12]:
aver_mean_lis5,F_measure_lis5=rfe_relief_model.featu_score(0.55,5)
print(aver_mean_lis5,F_measure_lis5)

[0.7922542930215564, 0.7789524505454356, 0.798873897385041, 0.8049976512344068, 0.8014236129234303, 0.8024414113471476, 0.8009068844929276, 0.7947661673364997, 0.7968135080119005, 0.7963046088000418, 0.7952881152460984, 0.7932420794404719, 0.7860835638603267, 0.7763870765697585, 0.7641317396523828, 0.7702541886319745, 0.7620909233258522, 0.7626050420168068, 0.7600540216086434, 0.7523957409050577, 0.7488177879847591, 0.7513727230022444, 0.7493266871966178, 0.7508625189206117, 0.7472871757398611, 0.7472871757398611, 0.7462654627068218, 0.7483062790333526, 0.7437026984706927, 0.7452359204551385, 0.7431964089983819, 0.7391121666057727, 0.7380930633122814, 0.7370674356699201, 0.7334933973589435, 0.7345138055222089, 0.7334881778798475, 0.7273670337700299, 0.7248094890129965, 0.7253223028341772, 0.7243018946709118, 0.7227712824260139, 0.7232801816378724, 0.7268555248186231, 0.726856829688397, 0.7227686726864659, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545, 

In [13]:
aver_mean_lis6,F_measure_lis6=rfe_relief_model.featu_score(0.65,5)
print(aver_mean_lis6,F_measure_lis6)

[0.7922542930215564, 0.7789524505454356, 0.798873897385041, 0.8049976512344068, 0.8014236129234303, 0.8024414113471476, 0.8009068844929276, 0.7947661673364997, 0.7968135080119005, 0.7963046088000418, 0.7952881152460984, 0.7932420794404719, 0.7860835638603267, 0.7763870765697585, 0.7641317396523828, 0.7702541886319745, 0.7620909233258522, 0.7626050420168068, 0.7600540216086434, 0.7523957409050577, 0.7488177879847591, 0.7513727230022444, 0.7493266871966178, 0.7508625189206117, 0.7472871757398611, 0.7472871757398611, 0.7462654627068218, 0.7483062790333526, 0.7437026984706927, 0.7452359204551385, 0.7431964089983819, 0.7391121666057727, 0.7380930633122814, 0.7370674356699201, 0.7334933973589435, 0.7345138055222089, 0.7334881778798475, 0.7273670337700299, 0.7248094890129965, 0.7253223028341772, 0.7243018946709118, 0.7227712824260139, 0.7232801816378724, 0.7268555248186231, 0.726856829688397, 0.7227686726864659, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545, 

In [14]:
aver_mean_lis7,F_measure_lis7=rfe_relief_model.featu_score(0.75,5)
print(aver_mean_lis7,F_measure_lis7)

[0.7922542930215564, 0.7789524505454356, 0.798873897385041, 0.8049976512344068, 0.8014236129234303, 0.8024414113471476, 0.8009068844929276, 0.7947661673364997, 0.7968135080119005, 0.7963046088000418, 0.7952881152460984, 0.7932420794404719, 0.7860835638603267, 0.7763870765697585, 0.7641317396523828, 0.7702541886319745, 0.7620909233258522, 0.7626050420168068, 0.7600540216086434, 0.7523957409050577, 0.7488177879847591, 0.7513727230022444, 0.7493266871966178, 0.7508625189206117, 0.7472871757398611, 0.7472871757398611, 0.7462654627068218, 0.7483062790333526, 0.7437026984706927, 0.7452359204551385, 0.7431964089983819, 0.7391121666057727, 0.7380930633122814, 0.7370674356699201, 0.7334933973589435, 0.7345138055222089, 0.7334881778798475, 0.7273670337700299, 0.7248094890129965, 0.7253223028341772, 0.7243018946709118, 0.7227712824260139, 0.7232801816378724, 0.7268555248186231, 0.7232762670285506, 0.7253170833550813, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545,

In [15]:
aver_mean_lis8,F_measure_lis8=rfe_relief_model.featu_score(0.85,5)
print(aver_mean_lis8,F_measure_lis8)

[0.7922542930215564, 0.7789524505454356, 0.798873897385041, 0.8049976512344068, 0.8014236129234303, 0.8024414113471476, 0.8009068844929276, 0.7947661673364997, 0.7968135080119005, 0.7963046088000418, 0.7952881152460984, 0.7932420794404719, 0.7860835638603267, 0.7763870765697585, 0.7641317396523828, 0.7610692102928128, 0.7605668354298241, 0.7626050420168068, 0.7600540216086434, 0.7523957409050577, 0.7488177879847591, 0.7513727230022444, 0.7493266871966178, 0.7508625189206117, 0.7472871757398611, 0.7472871757398611, 0.7462654627068218, 0.7483062790333526, 0.7437026984706927, 0.7452359204551385, 0.7431964089983819, 0.7391121666057727, 0.7380930633122814, 0.7370674356699201, 0.7334933973589435, 0.7345138055222089, 0.7334881778798475, 0.7273670337700299, 0.7248094890129965, 0.7253223028341772, 0.7243018946709118, 0.7227712824260139, 0.7232801816378724, 0.7268555248186231, 0.7232762670285506, 0.7253170833550813, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545,

In [16]:
aver_mean_lis9,F_measure_lis9=rfe_relief_model.featu_score(0.95,5)
print(aver_mean_lis9,F_measure_lis9)

[0.7922542930215564, 0.7789524505454356, 0.7758886163160916, 0.7947857403831098, 0.7927383997077091, 0.7963059136698157, 0.7947739965551439, 0.7952868103763244, 0.7912064825930372, 0.7876298345425127, 0.7866081215094733, 0.7809906571324181, 0.7871052768933661, 0.7835299337126156, 0.7763883814395323, 0.7646471632131113, 0.7575003914609322, 0.7508677383997078, 0.7498408058875723, 0.750351009969205, 0.7508586043112897, 0.7472845660003131, 0.7488230074638551, 0.750356229448301, 0.7467756667884544, 0.7462628529672739, 0.7437092228195625, 0.7442142074220993, 0.7411529829323034, 0.7442102928127772, 0.74012996502949, 0.7391082519964508, 0.7386006576543661, 0.74165796753484, 0.7391082519964508, 0.7360391982880108, 0.7334881778798475, 0.7273670337700299, 0.7294078500965603, 0.7278759329818885, 0.7288937314056056, 0.7294026306174645, 0.728382222454199, 0.7273618142909337, 0.7263361866485726, 0.7253170833550813, 0.7242966751918158, 0.7258285923064878, 0.7268463907302051] [0.8004204674229545, 0.792

In [28]:
np.max(aver_mean_lis1[:10]),np.max(aver_mean_lis2[:10]),np.max(aver_mean_lis3[:10]),np.max(aver_mean_lis4[:10]),np.max(aver_mean_lis5[:10])

(0.8049976512344068,
 0.8049976512344068,
 0.8049976512344068,
 0.8049976512344068,
 0.8049976512344068)

In [29]:
np.max(aver_mean_lis6[:10]),np.max(aver_mean_lis7[:10]),np.max(aver_mean_lis8[:10]),np.max(aver_mean_lis9[:10])

(0.8049976512344068,
 0.8049976512344068,
 0.8049976512344068,
 0.7963059136698157)

In [30]:
np.argmax(aver_mean_lis1[:10])

3

# SVM-RFE

In [18]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,folds):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            if len(candi_fea)==featur_num:
                return candi_fea
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                #print(scor_fea)
                rfe_score[k]=scor_fea
                k=k+1     
            #print(rfe_score)
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(1)

In [19]:
rfe_SVM_model=rfe_SVM(x_train,y_train,1.5,0.1)
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis=rfe_SVM_model.featu_score(5)

In [20]:
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis

([0.7922542930215564,
  0.7789524505454356,
  0.798873897385041,
  0.8049976512344068,
  0.8014236129234303,
  0.8024414113471476,
  0.8009068844929276,
  0.7947661673364997,
  0.7968135080119005,
  0.7963046088000418,
  0.7952881152460984,
  0.7932420794404719,
  0.7860835638603267,
  0.7763870765697585,
  0.7641317396523828,
  0.7702541886319745,
  0.7620909233258522,
  0.7626050420168068,
  0.7600540216086434,
  0.7523957409050577,
  0.7488177879847591,
  0.7513727230022444,
  0.7493266871966178,
  0.7508625189206117,
  0.7472871757398611,
  0.7472871757398611,
  0.7462654627068218,
  0.7483062790333526,
  0.7437026984706927,
  0.7452359204551385,
  0.7431964089983819,
  0.7391121666057727,
  0.7380930633122814,
  0.7370674356699201,
  0.7334933973589435,
  0.7345138055222089,
  0.7334881778798475,
  0.7273670337700299,
  0.7248094890129965,
  0.7253223028341772,
  0.7243018946709118,
  0.7268503053395271,
  0.7273644240304817,
  0.7268555248186231,
  0.726856829688397,
  0.72276867

# SVM-RELIEF

In [21]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class relief_SVM:
    def __init__(self,x_train,y_train,C,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
        self.Gamma=gamma
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        featu_num=self.X_train.shape[1]
        relief_fea_score=reliefF.reliefF(self.X_train,self.Y_train)
        relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        for i in range(featu_num):
            estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
            candi_fea=relief_candi_fea[:(i+1)]
            aver=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i]=aver
            F_lis[i]=F1 
        return aver_lis,F_lis

In [22]:
relief_SVM_model=relief_SVM(x_train,y_train,1.5,0.1)
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis=relief_SVM_model.featu_score(5)

In [23]:
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis

([0.7902108669554779,
  0.7983623884336344,
  0.7988673730361711,
  0.7993827965968997,
  0.8004032047601649,
  0.798873897385041,
  0.8039759382013676,
  0.7993762722480297,
  0.810612505871914,
  0.8100983871809593,
  0.807543452163474,
  0.8111266245628685,
  0.8101009969205073,
  0.8039694138524975,
  0.804989822015763,
  0.797323712093533,
  0.7952815908972284,
  0.7947687770760478,
  0.7860940028185187,
  0.788642413487134,
  0.7891513126989926,
  0.7814904222558589,
  0.787101362284044,
  0.7850566313481915,
  0.7855655305600501,
  0.7830132052821128,
  0.7758664335299337,
  0.7773957409050577,
  0.7748486351062164,
  0.7728039041703638,
  0.7768959757816171,
  0.7753588391878491,
  0.7758729578788037,
  0.7712680724463699,
  0.7676914243958454,
  0.7682068479565739,
  0.763612401482332,
  0.7615689754162533,
  0.7610561615950727,
  0.7661660316300434,
  0.7671864397933087,
  0.7615676705464794,
  0.7615689754162535,
  0.7630982827913774,
  0.7590114306592202,
  0.75441828905475

In [24]:
jinshi_accuracy=[]
jinshi_accuracy.append(aver_mean_lis1)
jinshi_accuracy.append(aver_mean_lis2)
jinshi_accuracy.append(aver_mean_lis3)
jinshi_accuracy.append(aver_mean_lis4)
jinshi_accuracy.append(aver_mean_lis5)
jinshi_accuracy.append(aver_mean_lis6)
jinshi_accuracy.append(aver_mean_lis7)
jinshi_accuracy.append(aver_mean_lis8)
jinshi_accuracy.append(aver_mean_lis9)
jinshi_accuracy.append(rfe_SVM_aver_mean_lis)
jinshi_accuracy.append(relief_SVM_aver_mean_lis)

In [25]:
colu=[i for i in range(1,50)]
jinshi_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(jinshi_accuracy))
jinshi_acurracy_dataframe.to_csv("jinshi_accuracy.csv")

In [26]:



jinshi_F_measure=[]
jinshi_F_measure.append(F_measure_lis1)
jinshi_F_measure.append(F_measure_lis2)
jinshi_F_measure.append(F_measure_lis3)
jinshi_F_measure.append(F_measure_lis4)
jinshi_F_measure.append(F_measure_lis5)
jinshi_F_measure.append(F_measure_lis6)
jinshi_F_measure.append(F_measure_lis7)
jinshi_F_measure.append(F_measure_lis8)
jinshi_F_measure.append(F_measure_lis9)
jinshi_F_measure.append(rfe_SVM_F_measure_lis)
jinshi_F_measure.append(relief_SVM_F_measure_lis)

In [27]:
colu=[i for i in range(1,50)]
jinshi_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(jinshi_F_measure))
jinshi_F_measure_dataframe.to_csv("jinshi_F_measure.csv")